In [1]:
import pandas as pd

df = pd.read_csv('dataset/P01/ActivityEvent.csv')

df['sum'] = df['confidenceStill'] + df['confidenceUnknown'] + df['confidenceOnFoot'] + df['confidenceInVehicle'] + df['confidenceOnBicycle'] + df['confidenceTilting']

print(df['sum'].mean())

df['sum']

0.9544534362549801


0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
8027    1.0
8028    1.0
8029    1.0
8030    1.0
8031    1.0
Name: sum, Length: 8032, dtype: float64